In [ ]:
import math
import pandas as pd
import plotly.graph_objects as go
import numpy as np
import os
import pandas_gbq


def round_sig_fig(num, sig_fig):
    return round(num, sig_fig - len(str(int(num))))

In [ ]:
active_validators =  1000000
base_reward_per_increment = 1e9*64//math.sqrt(32e9*active_validators)
base_reward_per_increment

In [ ]:
value_head_vote = 14/64*32*base_reward_per_increment
value_source_vote = 14/64*32*base_reward_per_increment
value_target_vote = 26/64*32*base_reward_per_increment
value_sum_attestation = sum(
    [value_head_vote,
    value_source_vote,
    value_target_vote]
)
value_sum_attestation

In [ ]:
value_sync_committee = 2/(32*512*64)*active_validators*32*base_reward_per_increment//1
value_sync_committee

In [ ]:
value_proposal_attestation = 8/56*value_sum_attestation*active_validators//32
value_proposal_sync = value_sync_committee*512*8//56
value_proposal_attestation, value_proposal_sync

In [ ]:
value_proposal_sync = 512*8/56*value_sync_committee//1
value_proposal_sync

In [ ]:
reward_attestation_epoch = value_sum_attestation*active_validators
reward_sync_committee_epoch = value_sync_committee*512*32
reward_proposer_epoch = 32*(value_proposal_attestation+value_proposal_sync)
epoch_reward = round_sig_fig(sum(
    [
        reward_attestation_epoch,
        reward_sync_committee_epoch,
        reward_proposer_epoch
    ]
)/1e9, 5)
epoch_reward

In [ ]:
reward_attestation_epoch/32/1e9

In [ ]:
reward_proposer_epoch/32/1e9

In [ ]:
(value_proposal_attestation+value_proposal_sync)/1e9

In [ ]:
(active_validators*32*base_reward_per_increment, # issued per epoch
active_validators*32*base_reward_per_increment/1e9)

In [ ]:
head = value_head_vote*active_validators
source = value_source_vote*active_validators
target = value_target_vote*active_validators
mevboost_value = 0.306*1e9*32
mevboost_value

In [ ]:
GOOGLE_CREDENTIALS = "..."

try:
    os.environ['GOOGLE_APPLICATION_CREDENTIALS']
except:
    print(f"setting google credentials as global variable...")
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_CREDENTIALS
    
df = pandas_gbq.read_gbq("""
SELECT distinct A.block_number, A.reward, B.slot, C.value from ...
""")

df = df.sort_values("slot").reset_index(drop=True)
df.to_parquet("mevboost_value_reward_data.parquet", index=False)
df

In [ ]:
df = pd.read_parquet("mevboost_value_reward_data.parquet")
df.loc[df["value"] != None, "value"] = df.loc[df["value"] != None, "value"].astype(float) / 1e18

local = df[df["value"].isna()]
nlocal = df[~df["value"].isna()]

local = local[["slot", "reward"]]
nlocal = nlocal[["slot", "value"]]

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=local['reward'],
    histnorm='percent',
    name='Local-building',
    opacity=0.75,
    nbinsx= 1500
))

fig.add_trace(go.Histogram(
    x=nlocal['value'],
    histnorm='percent',
    name='MEV Boost',
    opacity=0.75,
    nbinsx= 2400
    
))

fig.update_layout(
    title='Rewards: Local block building vs MEV-Boost',
    xaxis_title='Reward/Block (ETH)',
    yaxis_title='Percentage',
    barmode='overlay',
    height=550,
    legend=dict(
        x=0.87,
        y=0.99,
    ),
    width=1200,
    font=dict(
        family="Ubuntu Mono",
        size=14,
        color="black"
    ),
    plot_bgcolor = "#FFFFFF",
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
        title=None,
        range=[0, 0.18],
        tickvals=np.arange(0, 0.19, 0.01)
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgrey',
    ),
)

fig.write_image("images/reward_comparison.png")
fig.show()

In [ ]:
values = [value_sum_attestation*225*365/1e9, reward_proposer_epoch/32/1e9*(7200*365/active_validators), nlocal_filtered['value'].median()*(7200*365/active_validators)]
values2 = [value_sum_attestation*225*365/1e9, reward_proposer_epoch/32/1e9*(7200*365/active_validators), local_filtered['reward'].median()*(7200*365/active_validators)]

labels1 = ["attestation rewards", "proposer rewards", "el reward/mevboost"]
labels2 = ["attestation rewards", "proposer rewards", "el reward/mevboost"]

In [ ]:
colors = ['#377eb8', '#ff7f00', '#4daf4a', '#984ea3', '#f781bf', '#a65628', '#e41a1c', '#ffff33', '#a6cee3', '#999999']


fig = go.Figure()

fig.add_trace(go.Pie(
    labels=labels1,
    values=values,
    hole=0.3,
    domain=dict(x=[0, 0.5]),
    name="Pie Chart 1",
    marker=dict(colors=colors),
        sort=False

))

fig.add_trace(go.Pie(
    labels=labels2,
    values=values2,
    hole=0.3,
    domain=dict(x=[0.5, 1]),
    name="Pie Chart 2",
    marker=dict(colors=colors),
        sort=False

))

fig.update_layout(
    title='Reward Comparison: MEV Boost vs Local-building (1y)',
    margin={"t": 150},
    height=550,
    width=1200,
    font=dict(
        family="Ubuntu Mono",
        size=18,
        color="black"
    ),
    plot_bgcolor="#FFFFFF",
    annotations=[dict(text='With MEV-Boost', x=0.17, y=1.12, font_size=20, showarrow=False),
                 dict(text='With local building', x=0.85, y=1.12, font_size=20, showarrow=False)]
)

fig.write_image("images/rewards_comp_pie.png")
fig.show()